In [ ]:
# Importing Important Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
df = pd.read_csv('With-diff-Top4-Kerala-new.csv',)
df.head()

,Unnamed: 0,Kerala_Tourism_diff,Bus & Rail_diff,Travel Agencies_diff,Travel_diff,Kerala_Weather_diff,Target_diff
0,3,-0.086554,0.390932,-0.316707,0.608401,0.278507,-35825.0
1,7,-0.778985,0.097733,-0.633414,0.000000,0.000000,-1404.0
2,9,0.865539,-0.684130,0.791767,0.202800,0.278507,-4.0
3,12,-1.644523,-0.293199,-1.425180,-1.419602,-0.696267,45452.0
4,13,-0.346215,-0.781863,-0.475060,-0.608401,0.000000,-781.0


In [ ]:
x = df.drop(["Target_diff"], axis = 1)
y = df["Target_diff"]

In [ ]:
df = df.replace(to_replace="<1",
           value=0.5)

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from math import sqrt
# X = X
# y = y

# # X['SKU id'] = X['SKU id'].replace({'SKU-': ''}, regex=True)
# # X['Warehouse id'] = X['Warehouse id'].replace({'Wh-': ''}, regex=True)
# #X.head()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=False)

In [ ]:
import re
x_train = x_train.rename(columns = lambda u:re.sub('[^A-Za-z0-9_]+', '', u))
x_test = x_test.rename(columns = lambda u:re.sub('[^A-Za-z0-9_]+', '', u))
x = x.rename(columns = lambda u:re.sub('[^A-Za-z0-9_]+', '', u))

In [ ]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Step 1: Train Individual Models
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
lgbm_model = LGBMRegressor(n_estimators=100, random_state=42)
et_model = ExtraTreesRegressor(n_estimators=100, random_state=42)

# Assuming X_train, y_train, X_val, y_val are your training and validation sets
rf_model.fit(x_train, y_train)
lgbm_model.fit(x_train, y_train)
et_model.fit(x_train, y_train)

# Step 2: Make Predictions
rf_pred = rf_model.predict(x_test)
lgbm_pred = lgbm_model.predict(x_test)
et_pred = et_model.predict(x_test)

# Step 3: Calculate Average Predictions
ensemble_pred = np.mean([rf_pred, lgbm_pred, et_pred], axis=0)

# Step 4: Evaluate Performance
ensemble_mse = mean_squared_error(y_test, ensemble_pred)
print(f"Ensemble Mean Squared Error: {ensemble_mse}")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000261 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 121
[LightGBM] [Info] Number of data points in the train set: 110, number of used features: 6
[LightGBM] [Info] Start training from score 11792.763636
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [ ]:
mape = mean_absolute_percentage_error(y_test, ensemble_pred)

print(r2_score(y_test, ensemble_pred))
print(sqrt(mse(y_test,ensemble_pred)))
mape

0.5281494927433976
183916.89923165133


3.255084680323233

In [ ]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import numpy as np

# Step 1: Hyperparameter Tuning for Individual Models
rf_params = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}
lgbm_params = {'n_estimators': [50, 100, 200], 'max_depth': [5, 10, 15]}
et_params = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}

rf_model = GridSearchCV(RandomForestRegressor(random_state=42), rf_params, cv=3, scoring='neg_mean_squared_error')
lgbm_model = GridSearchCV(LGBMRegressor(random_state=42), lgbm_params, cv=3, scoring='neg_mean_squared_error')
et_model = GridSearchCV(ExtraTreesRegressor(random_state=42), et_params, cv=3, scoring='neg_mean_squared_error')

# Assuming X_train, y_train are your training sets
rf_model.fit(x_train, y_train)
lgbm_model.fit(x_train, y_train)
et_model.fit(x_train, y_train)

print("Best parameters for Random Forest:", rf_model.best_params_)
print("Best parameters for LightGBM:", lgbm_model.best_params_)
print("Best parameters for Extra Trees:", et_model.best_params_)

# Step 2: Make Predictions
rf_pred = rf_model.predict(x_test)
lgbm_pred = lgbm_model.predict(x_test)
et_pred = et_model.predict(x_test)

# Step 3: Calculate Average Predictions
ensemble_pred = np.mean([rf_pred, lgbm_pred, et_pred], axis=0)

# Step 4: Evaluate Performance
ensemble_mse = mean_squared_error(y_test, ensemble_pred)
print(f"Ensemble Mean Squared Error: {ensemble_mse}")

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000042 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 85
[LightGBM] [Info] Number of data points in the train set: 73, number of used features: 6
[LightGBM] [Info] Start training from score 4271.602740
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

In [ ]:
mape = mean_absolute_percentage_error(y_test, ensemble_pred)

print(r2_score(y_test, ensemble_pred))
print(sqrt(mse(y_test,ensemble_pred)))
mape

0.5164770249992396
186177.83706114767


3.252565940288742

In [ ]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import numpy as np

# Step 1: Hyperparameter Tuning for Individual Models
rf_params = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}
lgbm_params = {'n_estimators': [50, 100, 200], 'max_depth': [5, 10, 15]}
et_params = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}

rf_model = GridSearchCV(RandomForestRegressor(random_state=42), rf_params, cv=3, scoring='neg_mean_squared_error')
lgbm_model = GridSearchCV(LGBMRegressor(random_state=42), lgbm_params, cv=3, scoring='neg_mean_squared_error')
et_model = GridSearchCV(ExtraTreesRegressor(random_state=42), et_params, cv=3, scoring='neg_mean_squared_error')

# Assuming X_train, y_train are your training sets
rf_model.fit(x_train, y_train)
lgbm_model.fit(x_train, y_train)
et_model.fit(x_train, y_train)

print("Best parameters for Random Forest:", rf_model.best_params_)
print("Best parameters for LightGBM:", lgbm_model.best_params_)
print("Best parameters for Extra Trees:", et_model.best_params_)

# Step 2: Make Predictions
rf_pred = rf_model.predict(x_test)
lgbm_pred = lgbm_model.predict(x_test)
et_pred = et_model.predict(x_test)

# Step 3: Define Weights
rf_weight = 0.35135
lgbm_weight = 0.327
et_weight = 0.32165

# Step 4: Calculate Weighted Average Predictions
ensemble_pred = (rf_weight * rf_pred) + (lgbm_weight * lgbm_pred) + (et_weight * et_pred)

# Step 5: Evaluate Performance
ensemble_mse = mean_squared_error(y_test, ensemble_pred)
print(f"Ensemble Mean Squared Error: {ensemble_mse}")

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000050 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 85
[LightGBM] [Info] Number of data points in the train set: 73, number of used features: 6
[LightGBM] [Info] Start training from score 4271.602740
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

In [ ]:
mape = mean_absolute_percentage_error(y_test, ensemble_pred)

print(r2_score(y_test, ensemble_pred))
print(sqrt(mse(y_test,ensemble_pred)))
mape

0.5172314554311713
186032.53573674135


3.265007016798568

In [ ]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Step 1: Train Individual Models
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
lgbm_model = LGBMRegressor(n_estimators=100, random_state=42)
et_model = ExtraTreesRegressor(n_estimators=100, random_state=42)

# Assuming X_train, y_train, X_val, y_val are your training and validation sets
rf_model.fit(x_train, y_train)
lgbm_model.fit(x_train, y_train)
et_model.fit(x_train, y_train)

# Step 2: Make Predictions
rf_pred = rf_model.predict(x_test)
lgbm_pred = lgbm_model.predict(x_test)
et_pred = et_model.predict(x_test)

# Step 3: Define Weights
rf_weight = 0.35135
lgbm_weight = 0.327
et_weight = 0.32165

# Step 4: Calculate Weighted Average Predictions
ensemble_pred = (rf_weight * rf_pred) + (lgbm_weight * lgbm_pred) + (et_weight * et_pred)

# Step 5: Evaluate Performance
ensemble_mse = mean_squared_error(y_test, ensemble_pred)
print(f"Ensemble Mean Squared Error: {ensemble_mse}")

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000043 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 121
[LightGBM] [Info] Number of data points in the train set: 110, number of used features: 6
[LightGBM] [Info] Start training from score 11792.763636
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

In [ ]:
mape = mean_absolute_percentage_error(y_test, ensemble_pred)

print(r2_score(y_test, ensemble_pred))
print(sqrt(mse(y_test,ensemble_pred)))
mape

0.5284294954635671
183862.3216977282


3.267242713640996

In [ ]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Step 1: Train Individual Models
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
lgbm_model = LGBMRegressor(n_estimators=100, random_state=42)
et_model = ExtraTreesRegressor(n_estimators=100, random_state=42)

# Assuming X_train, y_train, X_val, y_val are your training and validation sets
rf_model.fit(x_train, y_train)
lgbm_model.fit(x_train, y_train)
et_model.fit(x_train, y_train)

# Step 2: Make Predictions
rf_pred = rf_model.predict(x_test)
lgbm_pred = lgbm_model.predict(x_test)
et_pred = et_model.predict(x_test)

# Step 3: Create Stacking Ensemble (Meta-Model)
meta_model = StackingRegressor(
    estimators=[('rf', rf_model), ('lgbm', lgbm_model), ('et', et_model)],
    final_estimator=LinearRegression(),  # You can use a different final estimator if needed
    cv=5  # Number of cross-validation folds for meta-model training
)

# Fit the meta-model using the predictions of the base models
meta_model.fit(np.column_stack((rf_pred, lgbm_pred, et_pred)), y_test)

# Step 4: Make Predictions with the Meta-Model
meta_pred = meta_model.predict(np.column_stack((rf_model.predict(x_test), lgbm_model.predict(x_test), et_model.predict(x_test))))

# Step 5: Evaluate Performance
ensemble_mse = mean_squared_error(y_test, meta_pred)
print(f"Ensemble Mean Squared Error: {ensemble_mse}")

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000053 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 121
[LightGBM] [Info] Number of data points in the train set: 110, number of used features: 6
[LightGBM] [Info] Start training from score 11792.763636
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

In [ ]:
mape = mean_absolute_percentage_error(y_test, ensemble_pred)

print(r2_score(y_test, ensemble_pred))
print(sqrt(mse(y_test,ensemble_pred)))
mape

0.5284294954635671
183862.3216977282


3.267242713640996

In [ ]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from lightgbm import LGBMRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import numpy as np

# Step 1: Define Base Models
rf_model = RandomForestRegressor(random_state=42)
lgbm_model = LGBMRegressor(random_state=42)
et_model = ExtraTreesRegressor(random_state=42)

# Step 2: Define Meta-Model (Random Forest Regressor)
meta_model = RandomForestRegressor(random_state=42)

# Step 3: Define Stacking Ensemble
ensemble = StackingCVRegressor(
    regressors=[rf_model, lgbm_model, et_model],
    meta_regressor=meta_model,
    cv=5
)

# Step 4: Train Stacking Ensemble
ensemble.fit(x_train, y_train)

# Step 5: Make Predictions
ensemble_pred = ensemble.predict(x_test)

# Step 6: Evaluate Performance
ensemble_mse = mean_squared_error(y_test, ensemble_pred)
print(f"Ensemble Mean Squared Error: {ensemble_mse}")

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000047 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 101
[LightGBM] [Info] Number of data points in the train set: 88, number of used features: 6
[LightGBM] [Info] Start training from score 4983.443182
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but ExtraTreesRegressor was fitted without feature names
  warnings.warn(


In [ ]:
mape = mean_absolute_percentage_error(y_test, ensemble_pred)

print(r2_score(y_test, ensemble_pred))
print(sqrt(mse(y_test,ensemble_pred)))
mape

0.30440676095082697
223304.30678417502


1.9568039120977738

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np

# Step 1: Define the Random Forest Regressor
rf_model = RandomForestRegressor(random_state=42)

# Step 2: Define the hyperparameter grid to search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Step 3: Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(x_train, y_train)

# Step 4: Get the Best Model
best_rf_model = grid_search.best_estimator_

# Step 5: Make Predictions
rf_pred = best_rf_model.predict(x)

# Step 6: Evaluate Performance
mse = mean_squared_error(y, rf_pred)
print(f"Random Forest Mean Squared Error: {mse}")

# Optional: Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)
rf_pred

Random Forest Mean Squared Error: 15335813197.1677
Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}


array([  -5749.83468929,   -1243.80733976,   -3225.63232067,
        -16469.01774226,  -56091.06081333,    6935.96138875,
          1286.16709541,  -11513.34622689,    -367.61648526,
        -33241.14649729,    5628.25132537,  -51871.58047177,
          5380.64538227,   17422.70402808,    2925.0134803 ,
        -25345.91697927,   -3732.47097077,    5708.74843649,
         -8292.05037889,    1103.2205168 ,    9936.76757405,
         23031.67685429,   -6369.79383689,    2795.7621328 ,
        -31586.3240662 ,   25680.44774702,  113419.70878207,
         74123.81237821,   64144.80505564,   50672.2365807 ,
         88269.02135445, -144149.53387994,   64947.33635374,
        104791.87547054,   82997.05054548,  112213.50520214,
        -41352.01783257,  137504.30345774, -158560.10404322,
        -96247.32907307,   88856.00368534,   71592.30246287,
        110988.23789166, -178499.29479165,   81837.33469386,
         91354.27908025,   88488.67745677,  -45936.08785408,
        -47221.23723905,

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np

# Step 1: Define the Extra Trees Regressor
et_model = ExtraTreesRegressor(random_state=42)

# Step 2: Define the hyperparameter grid to search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Step 3: Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(estimator=et_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(x_train, y_train)

# Step 4: Get the Best Model
best_et_model = grid_search.best_estimator_

# Step 5: Make Predictions
et_pred = best_et_model.predict(x)

# Step 6: Evaluate Performance
mse = mean_squared_error(y, et_pred)
print(f"Extra Trees Mean Squared Error: {mse}")

# Optional: Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)
et_pred

Extra Trees Mean Squared Error: 18014418690.90061
Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}


array([ 1.24062131e+04,  8.85092929e+03,  1.32608666e+04, -5.08662655e+04,
       -5.15163964e+04,  5.13948527e+03,  3.87202331e+04,  1.01961282e+03,
        8.96404550e+01, -3.67692940e+04,  2.43990943e+04, -2.30862266e+04,
        1.74607146e+04,  1.50508464e+04,  2.09366373e+04, -6.16129355e+04,
        1.85403949e+04,  1.46266465e+04,  5.22703005e+03,  7.54234654e+03,
        1.93732391e+04,  4.08479524e+04, -7.91864420e+03,  2.43607977e+04,
        1.19377900e+04,  4.44001450e+04,  8.27264246e+04,  2.07752424e+04,
        5.12630417e+04,  3.42791200e+04,  7.21239692e+04, -1.08585115e+05,
        3.81931034e+04,  7.51631880e+04,  5.45227132e+04,  8.54026330e+04,
       -4.77053396e+03,  5.44158526e+04, -1.52099572e+05, -7.20584706e+04,
        7.02713438e+04,  8.09038778e+04,  8.72323103e+04, -1.36899362e+05,
        7.05816590e+04,  7.81361688e+04,  7.69678246e+04, -3.10763025e+04,
       -1.27996140e+04,  8.63080917e+04, -1.17412890e+05, -1.25889221e+05,
        7.36985516e+04,  

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Step 1: Define the LightGBM Regressor
lgbm_model = LGBMRegressor(random_state=42)

# Step 2: Define the hyperparameter grid to search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'learning_rate': [0.001, 0.01, 0.1],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
}

# Step 3: Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(estimator=lgbm_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(x_train, y_train)

# Step 4: Get the Best Model
best_lgbm_model = grid_search.best_estimator_

# Step 5: Make Predictions
lgbm_pred = best_lgbm_model.predict(x)

# Step 6: Evaluate Performance
mse = mean_squared_error(y, lgbm_pred)
print(f"LightGBM Mean Squared Error: {mse}")

# Optional: Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)
lgbm_pred

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

array([ 42105.90625605,  42201.46842703,  20307.56856879, -85361.26724486,
       -92719.75287077,  34349.63003249,  37191.76301793,   7800.93463298,
          802.76506297, -92719.75287077,  42900.94660789, -92719.75287077,
        17686.95951211,  41708.1156584 ,   9038.66387538, -92719.75287077,
        16891.91916027,  40963.73918462,  15307.06319828,    802.76506297,
        36049.5959465 ,  42548.59514661,   1155.11652424,  41406.42807519,
       -84023.41044383,  37191.76301793,  34349.63003249,  34349.63003249,
        17686.95951211,   8243.62352354,  46178.98662271, -92719.75287077,
        25884.36750674,  33658.85370044,  53507.76682393,  47798.58323397,
        11380.06937143,  53539.47578053, -92719.75287077, -92719.75287077,
        33306.50223917,  52811.30574499,  37255.14653711, -92719.75287077,
        57554.99039477,  57554.99039477,  57554.99039477, -75623.91162501,
        10937.38048086,  63264.17398473, -81842.65442183, -92719.75287077,
        56661.37077003,  

In [ ]:
df_combined = x.copy()
df_combined['rf_pred'] = rf_pred
df_combined['et_pred'] = et_pred
df_combined['lgbm_pred'] = lgbm_pred
df_combined['target'] = y

In [ ]:
df_combined

,Unnamed0,Kerala_Tourism_diff,BusRail_diff,TravelAgencies_diff,Travel_diff,Kerala_Weather_diff,rf_pred,et_pred,lgbm_pred,target
0,3,-0.086554,0.390932,-0.316707,0.608401,0.278507,-5749.834689,12406.213144,42105.906256,-35825.0
1,7,-0.778985,0.097733,-0.633414,0.000000,0.000000,-1243.807340,8850.929291,42201.468427,-1404.0
2,9,0.865539,-0.684130,0.791767,0.202800,0.278507,-3225.632321,13260.866613,20307.568569,-4.0
3,12,-1.644523,-0.293199,-1.425180,-1.419602,-0.696267,-16469.017742,-50866.265534,-85361.267245,45452.0
4,13,-0.346215,-0.781863,-0.475060,-0.608401,0.000000,-56091.060813,-51516.396407,-92719.752871,-781.0
...,...,...,...,...,...,...,...,...,...,...
133,149,0.000000,-0.390932,0.158353,-0.202800,-1.114027,-70971.315971,-101258.730940,21632.973427,-331589.0
134,151,-0.086554,-0.097733,-0.158353,-0.202800,0.278507,136192.859320,-17813.528066,73369.487927,-98386.0
135,152,0.086554,0.000000,0.158353,0.101400,-1.392534,116820.928490,80535.899325,51189.491818,91652.0
136,153,0.000000,0.293199,0.000000,0.101400,0.000000,157572.906424,117462.236889,56997.254681,166992.0


In [ ]:
df_combined.to_csv("dataset_with_predictions.csv", index = False)

In [ ]:
X = df_combined.drop(["target"], axis = 1)
y = df_combined["target"]

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 1: Define the Random Forest Regressor
rf_model = RandomForestRegressor(random_state=42)

# Step 2: Define the hyperparameter grid to search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Step 3: Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Step 4: Get the Best Model
best_rf_model = grid_search.best_estimator_

# Step 5: Make Predictions on the Test Set
rf_pred = best_rf_model.predict(X_test)

# Step 6: Evaluate Performance
mse = mean_squared_error(y_test, rf_pred)
print(f"Random Forest Mean Squared Error on Test Set: {mse}")

# Optional: Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)
mape = mean_absolute_percentage_error(y_test, rf_pred)

print(r2_score(y_test, rf_pred))
mape

Random Forest Mean Squared Error on Test Set: 22840475933.846874
Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
0.6201100383184532


2.060338869630478